### remember to change model saving path!!!

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import os
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm
import random
import torch.nn as nn
from dataset import Thyroid_Dataset
from model import Eff_Unet
from HarDMSEG import HarDMSEG
from loss_metric import DiceLoss, IOU_score, StructureLoss, TverskyLoss
from LightMed.model.LightMed import LightMed
from PMFSNet.lib.models.PMFSNet import PMFSNet
from PMFSNet.lib.models.PMFSNet_FFT import PMFSNet_FFT
from hybrid_model_v3 import HybridSegModel
from helper import postprocess_logits_with_fill

/home/twszbak764/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/twszbak764/.local/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/twszbak764/.local/lib/python3.12/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [3]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [35]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import os
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms.functional as tx
import random
import cv2
from PIL import ImageEnhance
from skimage.exposure import match_histograms
import torchvision.transforms as T
import torchvision.transforms.functional as F
class Ge_nodule_Dataset(Dataset):
    def __init__(self, csv_file, transform, image_size, training = True):
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        self.image_size = image_size
        self.training = training
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_name = self.df["image_name"][idx]
        mask_name = self.df["mask_name"][idx]
       

        from_folder = "../ge_data/nodule"
        
        image_path = f"{from_folder}/images/{image_name}"
        nodule_mask_path = f"{from_folder}/true_nodule_masks/{mask_name}"
        gland_mask_path = f"{from_folder}/inference_gland(wo_nodule)_masks/{mask_name}"

        image = Image.open(image_path).convert("L")
        nodule_mask = Image.open(nodule_mask_path).convert("L")
        gland_mask = Image.open(gland_mask_path).convert("L")
        
        image_tensor, nodule_mask_tensor, gland_mask_tensor = self.transform(image, nodule_mask, gland_mask, self.image_size)
        nodule_mask_tensor = (nodule_mask_tensor > 0.5).float()
        gland_mask_tensor = (gland_mask_tensor > 0.5).float()
        
        if torch.sum(nodule_mask_tensor) == 0 or torch.sum(gland_mask_tensor) == 0:
            return [None]
        return image_tensor, nodule_mask_tensor, gland_mask_tensor

In [36]:
image_size = 128
batch_size = 1

def test_augmentation(image, nodule_mask, gland_mask, image_size):
    resize = T.Resize((image_size, image_size))
    image = resize(image)
    nodule_mask = resize(nodule_mask)
    gland_mask = resize(gland_mask)
        
    image_tensor = tx.to_tensor(image)
    nodule_mask_tensor = tx.to_tensor(nodule_mask)
    gland_mask_tensor = tx.to_tensor(gland_mask)

    # If standardization
    mean = image_tensor.mean()
    std = image_tensor.std()
    std = std if std > 0 else 1.0  # avoid division by zero
    image_tensor = (image_tensor - mean) / std
    return image_tensor, nodule_mask_tensor, gland_mask_tensor
def custom_collate_fn(batch):
    # print(batch)
    filtered_batch = [item for item in batch if item[0] is not None]
    if len(filtered_batch) == 0:
        return [None, None, None]
    return torch.utils.data.dataloader.default_collate(filtered_batch)
test_path = "../ge_data/nodule/new_test.csv"
test_dataset = Ge_nodule_Dataset(test_path, transform = test_augmentation, image_size = image_size, training = False)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, collate_fn=custom_collate_fn)

image, nodule_mask, gland_mask = next(iter(test_dataloader))

In [37]:
image.shape

torch.Size([1, 1, 128, 128])

In [38]:
len(test_dataset)

101

In [39]:
# Only calculate nodule loss, IOU, DICE, because there is no gland data in the testing set
def val(dataloader, model, device, nodule_post_process = False):
    total_IOU = 0
    total_DICE = 0

    total_nodule_IOU = 0
    total_nodule_DICE = 0

    total_gland_IOU = 0
    total_gland_DICE = 0

    IOU_arr = []
    
    model.train()
    model.to(device)
    count = 0
    for image, nodule_mask, gland_mask in tqdm(dataloader):
        if image == None:
            continue
        image, nodule_mask, gland_mask = image.to(device), nodule_mask.to(device), gland_mask.to(device)
        
        outputs = model(image)
        
        nodule_output = outputs[:, 0:1, :, :]
        gland_output = outputs[:, 1:2, :, :]

        if nodule_post_process:
            nodule_output = torch.sigmoid(nodule_output)
            nodule_output = nodule_output > 0.5
            gland_output = torch.sigmoid(gland_output)
            gland_output = gland_output > 0.5
            nodule_output = nodule_output & gland_output
            
            nodule_IOU = IOU_score(nodule_output, nodule_mask, need_act = False)
            gland_IOU = IOU_score(gland_output, gland_mask, need_act = False)
    
            dice_loss = DiceLoss(need_act = False)
            nodule_DICE = 1 - dice_loss(nodule_output, nodule_mask)
            gland_DICE = 1 - dice_loss(gland_output, gland_mask)
            
        else:
            nodule_IOU = IOU_score(nodule_output, nodule_mask)
            gland_IOU = IOU_score(gland_output, gland_mask)
    
            dice_loss = DiceLoss()
            nodule_DICE = 1 - dice_loss(nodule_output, nodule_mask)
            gland_DICE = 1 - dice_loss(gland_output, gland_mask)
            
        IOU = (nodule_IOU + gland_IOU) / 2
        DICE = (nodule_DICE + gland_DICE) / 2
        
        total_IOU += IOU.item()
        total_DICE += DICE.item()

        total_nodule_IOU += nodule_IOU.item()
        total_nodule_DICE += nodule_DICE.item()
        total_gland_IOU += gland_IOU.item()
        total_gland_DICE += gland_DICE.item()

        IOU_arr.append(nodule_IOU.item())
        count+=1
    return total_IOU/count, total_DICE/count, total_nodule_IOU / count, total_nodule_DICE / count, total_gland_IOU / count, total_gland_DICE / count, IOU_arr

    

In [48]:

name = "hybrid_model_ge(nodule_with_gland(wo_nodule))_from_scratch_v1_aug_space(0.9)_weight_decay(0.0005)"
inference_name = f"models/from_scratch/ge(nodule_with_gland)/{name}"
model = HybridSegModel(in_channels = 1, out_channels = 2, output_size = image_size, layers_num = 3)
checkpoint = torch.load(f"{inference_name}/best_checkpoint.pth")
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [49]:


total_IOU_val, total_DICE_val, total_nodule_IOU_val, total_nodule_DICE_val, total_gland_IOU_val, total_gland_DICE_val, IOU_arr = val(test_dataloader, model, "cuda", nodule_post_process = False)
print(f"val IOU : {total_IOU_val}, val DICE : {total_DICE_val}")
print(f"val nodule IOU : {total_nodule_IOU_val}, val nodule DICE : {total_nodule_DICE_val}")
print(f"val gland IOU : {total_gland_IOU_val}, val gland DICE : {total_gland_DICE_val}")
   

100%|██████████| 101/101 [00:02<00:00, 44.09it/s]

val IOU : 0.6548726567537477, val DICE : 0.7579587163901566
val nodule IOU : 0.6982729804545346, val nodule DICE : 0.7920251317543558
val gland IOU : 0.6114723368889035, val gland DICE : 0.7238923022062471


In [ ]:
test_dataset = Ge_nodule_Dataset(test_path, transform = test_augmentation, image_size = image_size, training = False)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, collate_fn=custom_collate_fn)
model.to("cpu")
for image, nodule_mask, gland_mask in tqdm(test_dataloader):
    if image != None:
        outputs = model(image)
        nodule_output = outputs[:, 0:1, :, :]
        gland_output = outputs[:, 1:2, :, :]
        
        nodule_output = torch.sigmoid(nodule_output)
        nodule_output = nodule_output > 0.9

        gland_output = postprocess_logits_with_fill(gland_output)
        gland_output = gland_output > 0.5
        
        nodule_mask = nodule_mask > 0.5
        gland_mask = gland_mask > 0.5

        
        plt.figure(figsize = (10, 10))
        plt.subplot(2,3,1)
        plt.imshow(image[0][0])
        plt.title("image")
        plt.subplot(2,3,2)
        plt.imshow(nodule_mask[0][0])
        plt.title("nodule mask")
        plt.subplot(2,3,3)
        plt.imshow(nodule_output[0][0].detach().numpy())
        plt.title("nodule pred")

        plt.subplot(2,3,4)
        plt.imshow(image[0][0])
        plt.title("image")
        plt.subplot(2,3,5)
        plt.imshow(gland_mask[0][0].detach().numpy())
        plt.title("gland mask")
        plt.subplot(2,3,6)
        plt.imshow(gland_output[0][0].detach().numpy())
        plt.title("gland pred")
        plt.show()